
### Examining racial discrimination in the US job market

#### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

#### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes.

#### Exercise
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Discuss statistical significance.

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

****

In [8]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

## What test is appropriate and does CLT apply?

### Because our sample population is over 30, we can presume that CLT applies correctly. I think the best test would be chi-square test of independence because we are dealing with independent categorical variables and proportions.

# Hypothesis setup:

### For this test we are going to be checking whether race has a significant impact on whether or not the candidate is called back for an interview. Specifically if the percentage of 'black' candidates who receive a call back is significantly different than the percentage of 'white' candidates. This would be a two tailed:
### P1 - P2 = 0
### P1 - P2 != 0 

In [4]:
# number of callbacks for black-sounding names
bcalback =  sum(data[data.race=='b'].call)
bcand = len(data[data.race=='b'])
p1 = bcalback / bcand 
n1 = bcand
print 'black callbacks: ',bcalback / bcand * 100
print 'bcand sample: ',bcand
# number of callbacks for white-sounding names
wcallback = sum(data[data.race=='w'].call)
wcand = len(data[data.race=='w'])
p2 = wcallback / wcand
n2 = wcand
print 'white callbacks: ',wcallback / wcand * 100
print 'wcand sample: ', wcand

black callbacks:  6.4476386037
bcand sample:  2435
white callbacks:  9.65092402464
wcand sample:  2435


# Confidence Interval for two sample populations

In [14]:
from scipy.stats import beta

def binom_interval(success, total, confint=0.95):
    quantile = (1 - confint) / 2.
    lower = beta.ppf(quantile, success, total - success + 1)
    upper = beta.ppf(1 - quantile, success + 1, total - success)
    return (lower, upper)

print 'black candidate confidence interval: ', binom_interval(bcalback, bcand)
print 'white candidate confidence itnerval: ', binom_interval(wcallback, wcand)

black candidate confidence interval:  (0.055044638648157666, 0.074971416855759476)
white candidate confidence itnerval:  (0.085065918572887936, 0.10893324180253894)


# Margin of error

In [23]:
from math import sqrt
def compute_moe(n, x, z=1.96):
    p = x/n
    phat = 1-p
    moe = z * sqrt((p*phat)/n)
    return moe

print 'black candidate margin of error: ',compute_moe(bcand, bcalback)
print 'white candidate margin of error: ',compute_moe(wcand, wcallback)

black candidate margin of error:  0.00975515802791
white candidate margin of error:  0.0117287814691


# Compute p-value

In [5]:
chi2, p, dof, expected = stats.chi2_contingency(([[bcalback, bcand],[wcallback, wcand]]))

In [27]:
def compute_significance(p):
    if p <= .05:
        print 'The P value: ',p, '< or equal to .05, reject null hypothesis'
    else:
        print 'The P value: ',p, '> .05, accept null hypothesis'
        
compute_significance(p)

The P value:  0.000185547471811 < or equal to .05, reject null hypothesis


# Discuss:

## Because we achieved a p-value of less than .05, we can deem that we should reject the null hypothesis and with 95% confdidence that the proportions of both races are indeed not equal.